In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib
import os

# --- 1. ROBUST DATA LOADING & CLEANING ---
file_path = r"~/Downloads/human_vital_signs_dataset_2024.csv"

print(f"📂 Loading: {file_path}...")
try:
    df = pd.read_csv(file_path)
    print(f"✅ Raw Data Loaded. Shape: {df.shape}")
    print(f"   Columns found: {list(df.columns)}")
except FileNotFoundError:
    print("❌ ERROR: File not found. Please check the path.")
    raise

# [CRITICAL FIX] Map the CSV column names to our Internal Standard
# This fixes the "KeyError" by renaming what exists to what we need.
column_mapping = {
    'Systolic Blood Pressure': 'Systolic BP',
    'Diastolic Blood Pressure': 'Diastolic BP',
    'Weight (kg)': 'Weight',
    'Height (m)': 'Height',
    'Body Temperature': 'Body Temperature',
    'Oxygen Saturation': 'Oxygen Saturation',
    'Heart Rate': 'Heart Rate',
    'Respiratory Rate': 'Respiratory Rate'
}
df = df.rename(columns=column_mapping)

# [CRITICAL FIX] Force-Calculate Derived Features
# Even if they exist, we recalculate to ensure they match our training logic.
print("⚙️ Engineering Features (BMI, MAP)...")
df['BMI'] = df['Weight'] / (df['Height']**2)
df['MAP'] = df['Diastolic BP'] + (1/3) * (df['Systolic BP'] - df['Diastolic BP'])

# Handle missing values (Drop rows with NaNs to prevent training errors)
df = df.dropna()

# [CRITICAL FIX] Ensure Target Variable Exists
if 'Risk Category' not in df.columns:
    print("⚠️ 'Risk Category' column missing. Generating based on clinical rules...")
    # 1 = High Risk, 0 = Low Risk
    def get_risk(row):
        if (row['Heart Rate'] > 100 or row['Oxygen Saturation'] < 94 or 
            row['Body Temperature'] > 37.8 or row['Systolic BP'] > 140):
            return 1
        return 0
    df['Risk Category'] = df.apply(get_risk, axis=1)
else:
    # Ensure it's numeric (0/1) not string ("High Risk")
    if df['Risk Category'].dtype == 'object':
         df['Risk Category'] = df['Risk Category'].apply(lambda x: 1 if 'High' in str(x) else 0)

print(f"✅ Data Ready for Training. Final Shape: {df.shape}")

# --- 2. TRAIN MODELS ---
features = ['Heart Rate', 'Respiratory Rate', 'Body Temperature', 'Oxygen Saturation', 
            'Systolic BP', 'Diastolic BP', 'BMI', 'MAP']
X = df[features]
y = df['Risk Category']

# Scale Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Model A: Classifier (Random Forest)
print("🌲 Training Random Forest Classifier...")
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_scaled, y)

# Model B: Autoencoder (PyTorch)
class RecoveryBrain(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.enc = nn.Sequential(nn.Linear(dim, 16), nn.ReLU(), nn.Linear(16, 8))
        self.dec = nn.Sequential(nn.Linear(8, 16), nn.ReLU(), nn.Linear(16, dim))
    def forward(self, x): return self.dec(self.enc(x))

brain = RecoveryBrain(len(features))
opt = torch.optim.Adam(brain.parameters(), lr=0.005)

# Train on "Healthy" data only (Label 0)
healthy_indices = (y == 0).values
if sum(healthy_indices) > 0:
    healthy_tensor = torch.FloatTensor(X_scaled[healthy_indices])
    print(f"🧠 Training Neural Network on {len(healthy_tensor)} healthy samples...")
    for _ in range(300):
        opt.zero_grad()
        loss = nn.MSELoss()(brain(healthy_tensor), healthy_tensor)
        loss.backward()
        opt.step()
else:
    print("⚠️ Warning: No 'Healthy' samples found. Training on full dataset (Suboptimal).")
    full_tensor = torch.FloatTensor(X_scaled)
    for _ in range(300):
        opt.zero_grad()
        loss = nn.MSELoss()(brain(full_tensor), full_tensor)
        loss.backward()
        opt.step()

# --- 3. SAVE EVERYTHING ---
joblib.dump(clf, 'risk_classifier.pkl')
joblib.dump(scaler, 'scaler.pkl')
torch.save(brain.state_dict(), 'recovery_brain.pth')
print("\n🎉 SUCCESS! Models Saved: 'risk_classifier.pkl', 'scaler.pkl', 'recovery_brain.pth'")

📂 Loading: ~/Downloads/human_vital_signs_dataset_2024.csv...
✅ Raw Data Loaded. Shape: (200020, 17)
   Columns found: ['Patient ID', 'Heart Rate', 'Respiratory Rate', 'Timestamp', 'Body Temperature', 'Oxygen Saturation', 'Systolic Blood Pressure', 'Diastolic Blood Pressure', 'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Derived_HRV', 'Derived_Pulse_Pressure', 'Derived_BMI', 'Derived_MAP', 'Risk Category']
⚙️ Engineering Features (BMI, MAP)...
✅ Data Ready for Training. Final Shape: (200020, 19)
🌲 Training Random Forest Classifier...
🧠 Training Neural Network on 94905 healthy samples...

🎉 SUCCESS! Models Saved: 'risk_classifier.pkl', 'scaler.pkl', 'recovery_brain.pth'


In [2]:
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 MB 5.8 MB/s eta 0:00:0000:0100:01
